<a href="https://colab.research.google.com/github/ArshanBhanage/Autogluon/blob/main/California_Housing_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install -U pip setuptools wheel kaggle
!pip -q install -U autogluon.tabular

import importlib.metadata as im, sys, platform, pandas as pd
print("AutoGluon Tabular:", im.version('autogluon.tabular'))
print("Pandas:", pd.__version__)
print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 82.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
AutoGluon Tabular: 1.4.0
Pandas: 2.2.2
Python: 3.12.12
Platform: Linux-6.6.105+-x86_64-with-glibc2.35


In [11]:
!pip -q install kaggle

!kaggle config view
!kaggle competitions files -c california-house-prices
!kaggle competitions download -c california-house-prices


Configuration values from /root/.kaggle
- username: arshanbhanage
- path: None
- proxy: None
- competition: None
name                         size  creationDate                
---------------------  ----------  --------------------------  
sample_submission.csv      253022  2021-03-19 22:16:47.263000  
test.csv                 36182130  2021-03-19 22:16:47.263000  
train.csv                53361674  2021-03-19 22:16:47.263000  
  0% 0.00/29.5M [00:00<?, ?B/s]
100% 29.5M/29.5M [00:00<00:00, 1.80GB/s]


In [13]:
!unzip /content/california-house-prices.zip -d /content/

Archive:  /content/california-house-prices.zip
  inflating: /content/sample_submission.csv  
  inflating: /content/test.csv       
  inflating: /content/train.csv      


In [15]:
import pandas as pd
import os

sample_path = "/content/sample_submission.csv"
test_path  = "/content/test.csv"
train_path   = "/content/train.csv"

sample = pd.read_csv(sample_path)
train  = pd.read_csv(train_path, low_memory=False)
test   = pd.read_csv(test_path,  low_memory=False)

print("Files loaded:")
print(" sample_submission shape:", sample.shape)
print(" train shape:", train.shape)
print(" test  shape:", test.shape)


id_col, target_col = list(sample.columns)
print(f"\nInferred ID column: {id_col}")
print(f"Inferred TARGET column: {target_col}")

missing_in_train = [c for c in [id_col, target_col] if c not in train.columns]
if missing_in_train:
    raise ValueError(f"These columns from sample_submission are not in train.csv: {missing_in_train}")

if id_col not in test.columns:
    raise ValueError(f"ID column '{id_col}' not found in test.csv")

display(train[[id_col, target_col]].head())
display(test[[id_col]].head())

Files loaded:
 sample_submission shape: (31626, 2)
 train shape: (47439, 41)
 test  shape: (31626, 40)

Inferred ID column: Id
Inferred TARGET column: Sold Price


,Id,Sold Price
0,0,3825000.0
1,1,505000.0
2,2,140000.0
3,3,1775000.0
4,4,1175000.0


,Id
0,47439
1,47440
2,47441
3,47442
4,47443


In [16]:
import numpy as np

LABEL = target_col
ID    = id_col

feature_cols = [c for c in train.columns if c not in [LABEL, ID]]


missing_in_test = [c for c in feature_cols if c not in test.columns]
for c in missing_in_test:
    test[c] = np.nan

X_train = train[feature_cols].copy()
y_train = train[LABEL].copy()
X_test  = test[feature_cols].copy()


test_ids = test[ID].copy()

print("Feature columns:", len(feature_cols))
print("Train X shape:", X_train.shape, "| y shape:", y_train.shape)
print("Test  X shape:", X_test.shape)
print("Missing in test:", missing_in_test[:10], "… total:", len(missing_in_test))

dtypes_summary = X_train.dtypes.value_counts()
print("\nDtype counts in X_train:\n", dtypes_summary)
display(X_train.head())

Feature columns: 39
Train X shape: (47439, 39) | y shape: (47439,)
Test  X shape: (31626, 39)
Missing in test: [] … total: 0

Dtype counts in X_train:
 object     21
float64    17
int64       1
Name: count, dtype: int64


,Address,Summary,Type,Year built,Heating,Cooling,Parking,Lot,Bedrooms,Bathrooms,...,Parking features,Tax assessed value,Annual tax amount,Listed On,Listed Price,Last Sold On,Last Sold Price,City,Zip,State
0,540 Pine Ln,"540 Pine Ln, Los Altos, CA 94022 is a single f...",SingleFamily,1969.0,"Heating - 2+ Zones, Central Forced Air - Gas","Multi-Zone, Central AC, Whole House / Attic Fan","Garage, Garage - Attached, Covered",1.0,"Ground Floor Bedroom, Master Bedroom on Ground...",0.0,...,"Garage, Garage - Attached, Covered",886486.0,12580.0,2019-10-24,4198000.0,NaN,NaN,Los Altos,94022,CA
1,1727 W 67th St,"HURRY, HURRY.......Great house 3 bed and 2 bat...",SingleFamily,1926.0,Combination,"Wall/Window Unit(s), Evaporative Cooling, See ...","Detached Carport, Garage",4047.0,3,2.0,...,"Detached Carport, Garage",505000.0,6253.0,2019-10-16,525000.0,2019-08-30,328000.0,Los Angeles,90047,CA
2,28093 Pine Ave,'THE PERFECT CABIN TO FLIP! Strawberry deligh...,SingleFamily,1958.0,Forced air,NaN,0 spaces,9147.0,2,3.0,...,NaN,49627.0,468.0,2019-08-25,180000.0,NaN,NaN,Strawberry,95375,CA
3,10750 Braddock Dr,Rare 2-story Gated 5 bedroom Modern Mediterran...,SingleFamily,1947.0,Central,Central Air,"Detached Carport, Driveway, Garage - Two Door",NaN,5,3.0,...,"Detached Carport, Driveway, Garage - Two Door",1775000.0,20787.0,2019-10-24,1895000.0,2016-08-30,1500000.0,Culver City,90230,CA
4,7415 O Donovan Rd,Beautiful 200 acre ranch land with several pas...,VacantLand,NaN,NaN,NaN,0 spaces,NaN,NaN,NaN,...,NaN,NaN,NaN,2019-06-07,1595000.0,2016-06-27,900000.0,Creston,93432,CA


In [17]:
from autogluon.tabular import TabularPredictor


directory  = "/content/calhouse"
label      = LABEL
eval_metric= "root_mean_squared_error"
save_path  = "/content/AutogluonModels-CalHouse"

train_data = pd.concat([X_train, y_train], axis=1)

predictor = TabularPredictor(
    label=label,
    eval_metric=eval_metric,
    path=save_path,
    verbosity=3,
    problem_type="regression"
).fit(
    train_data=train_data,
    presets="good",
    time_limit=10800 # Increased time limit to 3600 seconds (1 hour)
)

results = predictor.fit_summary()
results

Preset alias specified: 'good' maps to 'good_quality'.
Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
GPU Count:          1
Memory Avail:       11.18 GB / 12.67 GB (88.2%)
Disk Space Avail:   73.30 GB / 112.64 GB (65.1%)
Presets specified: ['good']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True,
 'num_bag_sets': 1,
 'refit_full': True,
 'save_bag_folds': False,
 'set_best_to_refit_full': True}
Full kwargs:
{'_experimental_dynamic_hyperparameters': False,
 '_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'delay_bag_sets': False,
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
      

[50]	valid_set's rmse: 896568
[100]	valid_set's rmse: 766101
[150]	valid_set's rmse: 719135
[200]	valid_set's rmse: 699027
[250]	valid_set's rmse: 692261
[300]	valid_set's rmse: 688115
[350]	valid_set's rmse: 685761
[400]	valid_set's rmse: 687688
[450]	valid_set's rmse: 687680


	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05, 'extra_trees': True}


[50]	valid_set's rmse: 1.50915e+06
[100]	valid_set's rmse: 1.4505e+06
[150]	valid_set's rmse: 1.43154e+06
[200]	valid_set's rmse: 1.42502e+06
[250]	valid_set's rmse: 1.42069e+06
[300]	valid_set's rmse: 1.42039e+06
[350]	valid_set's rmse: 1.41971e+06
[400]	valid_set's rmse: 1.41864e+06
[450]	valid_set's rmse: 1.41822e+06
[500]	valid_set's rmse: 1.41796e+06
[550]	valid_set's rmse: 1.41889e+06
[600]	valid_set's rmse: 1.41944e+06
[650]	valid_set's rmse: 1.41857e+06


	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05, 'extra_trees': True}


...

	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05}


[50]	valid_set's rmse: 595001
[100]	valid_set's rmse: 558622
[150]	valid_set's rmse: 553217
[200]	valid_set's rmse: 555606


Saving /content/AutogluonModels-CalHouse/models/LightGBM_BAG_L1/utils/oof.pkl
Saving /content/AutogluonModels-CalHouse/models/LightGBM_BAG_L1/model.pkl
	-702586.1248	 = Validation score   (-root_mean_squared_error)
	244.38s	 = Training   runtime
	3.07s	 = Validation runtime
	1931.8	 = Inference  throughput (rows/s | 5930 batch size)
Saving /content/AutogluonModels-CalHouse/models/trainer.pkl
Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 2062.93s of the 3402.97s of remaining time.
	Fitting RandomForestMSE_BAG_L1 with 'num_gpus': 0, 'num_cpus': 2
Saving /content/AutogluonModels-CalHouse/models/RandomForestMSE_BAG_L1/utils/model_template.pkl
Loading: /content/AutogluonModels-CalHouse/models/RandomForestMSE_BAG_L1/utils/model_template.pkl
	8.9s	= Estimated out-of-fold prediction time...
	`use_child_oof` was specified for this model. It will function similarly to a bagged model, but will only fit one child model.
Saving /content/AutogluonModels-CalHouse/models/RandomFor

*** Summary of fit() ***
Estimated performance of each model:
                         model      score_val              eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0          WeightedEnsemble_L3 -685662.702262  root_mean_squared_error     110.371852  5843.122480                0.001142           0.037224            3      False          5
1          WeightedEnsemble_L2 -685662.702262  root_mean_squared_error     110.371965  5843.122006                0.001255           0.036750            2      False          4
2              LightGBM_BAG_L1 -702586.124850  root_mean_squared_error       3.069642   244.377477                3.069642         244.377477            1      False          2
3       RandomForestMSE_BAG_L1 -734401.243965  root_mean_squared_error     101.656322  5253.288066              101.656322        5253.288066            1       True          3
4            LightGBMXT_BAG_L1 -795557.352999  root_m

{'model_types': {'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForestMSE_BAG_L1': 'StackerEnsembleModel_RF',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel',
  'WeightedEnsemble_L3': 'WeightedEnsembleModel',
  'LightGBMXT_BAG_L1_FULL': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1_FULL': 'StackerEnsembleModel_LGB',
  'RandomForestMSE_BAG_L1_FULL': 'StackerEnsembleModel_RF',
  'WeightedEnsemble_L2_FULL': 'WeightedEnsembleModel',
  'WeightedEnsemble_L3_FULL': 'WeightedEnsembleModel'},
 'model_performance': {'LightGBMXT_BAG_L1': np.float64(-795557.3529990613),
  'LightGBM_BAG_L1': np.float64(-702586.1248495487),
  'RandomForestMSE_BAG_L1': np.float64(-734401.2439646777),
  'WeightedEnsemble_L2': np.float64(-685662.7022622661),
  'WeightedEnsemble_L3': np.float64(-685662.7022622661),
  'LightGBMXT_BAG_L1_FULL': None,
  'LightGBM_BAG_L1_FULL': None,
  'RandomForestMSE_BAG_L1_FULL': None,
  'WeightedEnsemble_L2_FULL': None,
  'We

In [18]:
# Make predictions on the test data
predictions = predictor.predict(X_test)

# Create a submission DataFrame
submission = pd.DataFrame({ID: test_ids, LABEL: predictions})

# Display the first few rows of the submission file
display(submission.head())

Loading: /content/AutogluonModels-CalHouse/models/LightGBMXT_BAG_L1_FULL/model.pkl
Loading: /content/AutogluonModels-CalHouse/models/LightGBM_BAG_L1_FULL/model.pkl
Loading: /content/AutogluonModels-CalHouse/models/RandomForestMSE_BAG_L1_FULL/model.pkl
Loading: /content/AutogluonModels-CalHouse/models/WeightedEnsemble_L3_FULL/model.pkl


,Id,Sold Price
0,47439,8.084541e+05
1,47440,5.761956e+05
2,47441,8.163788e+05
3,47442,8.158966e+05
4,47443,1.139827e+06
